# Working with Features using the Dseqrecord class

> Visit the full library documentation [here](https://bjornfjohansson.github.io/pydna/)

Features are important components of a .gb file, describing the key biological properties of the sequence. In Genbank, features "include genes, gene products, as well as regions of biological significance reported in the sequence." (See [here](https://www.ncbi.nlm.nih.gov/genbank/samplerecord/) for a description of a Genbank file and associated terminologies/annotations) Examples include coding sequences (CDS), introns, promoters, etc.

pydna offers many ways to easily view, add, extract, and write features into a Genbank file via the `Dseqrecord` class. Before working with features, you need to import the Genbank files (or other biological formats including FASTA, snapgene, EMBL) into python. Please refer to the Importing_Seqs page for a quick how-to tutorial.

After importing the file, we can check out the list of features in a sequence using the following code. An example `Dseqrecord` object has been loaded using the GenBank sample sequence [here](https://www.ncbi.nlm.nih.gov/genbank/samplerecord/).

In [ ]:
from pydna.dseqrecord import Dseqrecord
from pydna.parsers import parse

#Import your file into python. 
file_path = "./sequence.gb"
records = parse(file_path)
my_record = records[0]

# List all features
for feature in my_record.features:
    print(feature)

type: source
location: [0:5028](+)
qualifiers:
    Key: chromosome, Value: ['IX']
    Key: db_xref, Value: ['taxon:4932']
    Key: mol_type, Value: ['genomic DNA']
    Key: organism, Value: ['Saccharomyces cerevisiae']

type: mRNA
location: [<0:>206](+)
qualifiers:
    Key: product, Value: ['TCP1-beta']

type: CDS
location: [<0:206](+)
qualifiers:
    Key: codon_start, Value: ['3']
    Key: product, Value: ['TCP1-beta']
    Key: protein_id, Value: ['AAA98665.1']
    Key: translation, Value: ['SSIYNGISTSGLDLNNGTIADMRQLGIVESYKLKRAVVSSASEAAEVLLRVDNIIRARPRTANRQHM']

type: gene
location: [<686:>3158](+)
qualifiers:
    Key: gene, Value: ['AXL2']

type: mRNA
location: [<686:>3158](+)
qualifiers:
    Key: gene, Value: ['AXL2']
    Key: product, Value: ['Axl2p']

type: CDS
location: [686:3158](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: gene, Value: ['AXL2']
    Key: note, Value: ['plasma membrane glycoprotein']
    Key: product, Value: ['Axl2p']
    Key: protein_id, Value: ['AA

Additional ways to view and search for particular features are shown at the bottom of the page under "Other Methods to Viewing Features"

## Adding Features and Qualifiers

Adding a new feature to describe a region of interest, for instance for a region that you would like to perform a PCR, to a file involves the parent module `Bio.SeqFeature`. `Bio.SeqFeature` provides the `FeatureLocation` method to add a feature to a file, given the starting base, ending base, and the feature type. Note that the base numbering follow biological convention, rather than python numbering methods. For instance, the following code adds a new feature from the 2nd to the 5th nucleotide, of the mRNA type.

In [ ]:
from Bio.SeqFeature import FeatureLocation, SeqFeature

# Define the locations of the CDS
location = FeatureLocation(2, 5)

# Create a SeqFeature with the type mRNA
my_feature = SeqFeature(location=location, type="mRNA")

# Add my_feature to my_record with .append
my_record.features.append(my_feature)

# Confirm that my_feature has been added
my_record.features[-1]

SeqFeature(SimpleLocation(ExactPosition(2), ExactPosition(5)), type='mRNA')

Note that a new feature is always added to the last position of the features list.  

`pydna` and `Bio.SeqFeature` suppports all the conventional feature types through the `type` parameters. A non-exhaustive list include gene, CDS, promoter, exon, intron, 5' UTR, 3' UTR, terminator, enhancer, and RBS. You can also define custom features, which could be useful for synthetic biology applications. For instance, you might want to have Bio_brick or spacer features to describe a synthetic standardised plasmid construct.  
  
It is important to note that while `pydna` and `Bio.SeqFeature` does not restrict the feature types you can use, sticking to standard types helps maintain compatibility with other bioinformatics tools and databases. I recommend referring to the official [GenBank_Feature_Table](https://www.insdc.org/submitting-standards/feature-table/#2), if in doubt.

To make a note of what this sequence is, we can add a qualifier to the new feature by accessing the `qualifiers` dictionary. This dictionary can be accessed writing your notes as a dictionary key value pair, under the `qualifiers` parameter of the `SeqFeature` class object.  
For instance, if I would like to note a new feature of type 'domain', between 24-56 bases as my region of interest, I can instantiate the `SeqFeature` class object as such.

In [ ]:
location = FeatureLocation(24, 56)

# Create a SeqFeature with a qualifier
my_feature2 = SeqFeature(location=location, type="domain", qualifiers={"Note":"Region of interest"})

# Add my_feature to my_record with .append
my_record.features.append(my_feature2)

# Confirm that my_feature has been added
my_record.features[-1]

SeqFeature(SimpleLocation(ExactPosition(24), ExactPosition(56)), type='domain', qualifiers=...)

On the feature list of the .gb file, the new feature is reflected as so:

type: CDS  
location: [24:56]  
qualifiers:  
    Key: Note, Value: Region of interest  

Note that `Bio.SeqFeatures` does not automatically assume a sequence strand. If you would like to refer to a sequence on the positive or minus strand, you can add a parameter in `FeatureLocation` specifying `strand=+1` or `strand=-1`. 

In [ ]:
#Create a location specifying the minus strand
location = FeatureLocation(134, 520, strand=-1)

my_feature3 = SeqFeature(location=location, type="CDS", qualifiers={"gene":"example_CDS"})

my_record.features.append(my_feature3)

my_record.features[-1]

SeqFeature(SimpleLocation(ExactPosition(134), ExactPosition(520), strand=-1), type='CDS', qualifiers=...)

### Adding a Feature with Parts

To add a feature with parts, like a CDS with introns, we need to apply another class and methods from `Bio.SeqFeature`. Specifically, you can achieve this by creating a `CompoundLocation` object, and then adding it to a `SeqFeature` object as normal. 

The example code belows adds a CDS with two parts, between 5-15bp and 20-30bp, named "example gene" in the qualifiers, to my features list. 

In [ ]:
from pydna.dseqrecord import Dseqrecord
from Bio.SeqFeature import SeqFeature, FeatureLocation, CompoundLocation

# Define the locations of the CDS
locations = [FeatureLocation(5, 15), FeatureLocation(20, 30)]

# Create a compound location from these parts
compound_location = CompoundLocation(locations)

# Create a SeqFeature with this compound location, including type and qualifiers. 
cds_feature = SeqFeature(location=compound_location, type="CDS", qualifiers={"gene": "example_gene"})

# Add the feature to the Dseqrecord
my_record.features.append(cds_feature)

my_record.features[-1]

SeqFeature(CompoundLocation([SimpleLocation(ExactPosition(5), ExactPosition(15)), SimpleLocation(ExactPosition(20), ExactPosition(30))], 'join'), type='CDS', qualifiers=...)

  Our added feature looks like this on the features list, as appropriate:  

type: CDS  
location: join{[5:15], [20:30]} . 
qualifiers:  
    Key: gene, Value: example_gene  

Further documentation for `SeqFeature`, `CompoundLocation`, and `FeatureLocation` can be found in the `SeqFeature` module [here](https://biopython.org/docs/1.75/api/Bio.SeqFeature.html). 

### Handling Origin Spanning Features

An origin spanning feature is a special type of features that crosses over a circular sequence's origin. In pydna, such a feature is represented as a feature with parts, joining the part of the sequence before the origin and after the origin. They can be added using `CompoundLocation` as normal. 

An origin spanning feature, between base 19 to base 6, in a 25bp long circular sequence, is represented like so:   
  
type: gene 
location: join{[19:25](+), [0:6](+)}  
qualifiers: gene, Value: example_gene  
  
The code uses the `add_feature` method as normal.

In [ ]:
location = [FeatureLocation(4023, 4037), FeatureLocation(0, 6)]
ori_feat_location = CompoundLocation(location)
ori_feature = SeqFeature(location=ori_feat_location, type="misc", qualifiers={"gene":"example origin spanning gene"})
my_record.features.append(ori_feature)
my_record.features[-1]

SeqFeature(CompoundLocation([SimpleLocation(ExactPosition(4023), ExactPosition(4037)), SimpleLocation(ExactPosition(0), ExactPosition(6))], 'join'), type='misc', qualifiers=...)

### Other Methods to Viewing Features

pydna also provides the `list_features` method as a simple way to list all the features in a `Dseqrecord` object. 

In [ ]:
print(my_record.list_features())

+-----+------------------+-----+-------+-------+------+--------+------+
| Ft# | Label or Note    | Dir | Sta   | End   |  Len | type   | orf? |
+-----+------------------+-----+-------+-------+------+--------+------+
|   0 | nd               | --> | 0     | 5028  | 5028 | source |  no  |
|   1 | nd               | --> | <0    | >206  |  206 | mRNA   |  no  |
|   2 | nd               | --> | <0    | 206   |  206 | CDS    |  no  |
|   3 | nd               | --> | <686  | >3158 | 2472 | gene   | yes  |
|   4 | nd               | --> | <686  | >3158 | 2472 | mRNA   | yes  |
|   5 | N:plasma membran | --> | 686   | 3158  | 2472 | CDS    | yes  |
|   6 | nd               | <-- | <3299 | >4037 |  738 | gene   | yes  |
|   7 | nd               | <-- | <3299 | >4037 |  738 | mRNA   | yes  |
|   8 | nd               | <-- | 3299  | 4037  |  738 | CDS    | yes  |
|   9 | nd               | --- | 2     | 5     |    3 | mRNA   |  no  |
|  10 | nd               | --- | 24    | 56    |   32 | domain |

This method is convenient for checking-out a brief overview of each feature, without reading through an entire sequence record.  

Alternatively, we can look for specific features using their qualifiers. For instance, if I want to find my feature with the gene name of example_gene (This was the example feature for `CompoundLocation`), I can use the following code:

In [ ]:
gene = [f for f in my_record.features if "gene" in f.qualifiers and f.qualifiers["gene"] == "example_gene"]
print(gene)

[SeqFeature(CompoundLocation([SimpleLocation(ExactPosition(5), ExactPosition(15)), SimpleLocation(ExactPosition(20), ExactPosition(30))], 'join'), type='CDS', qualifiers=...)]


If you would like to search for another type of features, simply replace the `"gene"` with your desired feature type in quotation marks.

### Removing Features

In pydna, we can search for the feature that we would like to remove using the feature's types or qualififers. For instance, we can modify the features list to exclude all CDS:

In [ ]:
#Remove all CDS type features from my feature list
my_record.features = [f for f in my_record.features if not (f.type == "CDS")]

for feature in my_record.features:
    print(feature)

type: source
location: [0:5028](+)
qualifiers:
    Key: chromosome, Value: ['IX']
    Key: db_xref, Value: ['taxon:4932']
    Key: mol_type, Value: ['genomic DNA']
    Key: organism, Value: ['Saccharomyces cerevisiae']

type: mRNA
location: [<0:>206](+)
qualifiers:
    Key: product, Value: ['TCP1-beta']

type: gene
location: [<686:>3158](+)
qualifiers:
    Key: gene, Value: ['AXL2']

type: mRNA
location: [<686:>3158](+)
qualifiers:
    Key: gene, Value: ['AXL2']
    Key: product, Value: ['Axl2p']

type: gene
location: [<3299:>4037](-)
qualifiers:
    Key: gene, Value: ['REV7']

type: mRNA
location: [<3299:>4037](-)
qualifiers:
    Key: gene, Value: ['REV7']
    Key: product, Value: ['Rev7p']

type: mRNA
location: [2:5]
qualifiers:

type: domain
location: [24:56]
qualifiers:
    Key: Note, Value: Region of interest

type: misc
location: join{[4023:4037], [0:6]}
qualifiers:
    Key: gene, Value: example origin spanning gene



We can also modify the features list to exclude a specific gene:

In [ ]:
#Exclude example_gene from my feature list
my_record.features = [f for f in my_record.features if not (f.qualifiers == {"gene": "example_gene"})]

for feature in my_record.features:
    print(feature)

type: source
location: [0:5028](+)
qualifiers:
    Key: chromosome, Value: ['IX']
    Key: db_xref, Value: ['taxon:4932']
    Key: mol_type, Value: ['genomic DNA']
    Key: organism, Value: ['Saccharomyces cerevisiae']

type: mRNA
location: [<0:>206](+)
qualifiers:
    Key: product, Value: ['TCP1-beta']

type: gene
location: [<686:>3158](+)
qualifiers:
    Key: gene, Value: ['AXL2']

type: mRNA
location: [<686:>3158](+)
qualifiers:
    Key: gene, Value: ['AXL2']
    Key: product, Value: ['Axl2p']

type: gene
location: [<3299:>4037](-)
qualifiers:
    Key: gene, Value: ['REV7']

type: mRNA
location: [<3299:>4037](-)
qualifiers:
    Key: gene, Value: ['REV7']
    Key: product, Value: ['Rev7p']

type: mRNA
location: [2:5]
qualifiers:

type: domain
location: [24:56]
qualifiers:
    Key: Note, Value: Region of interest

type: misc
location: join{[4023:4037], [0:6]}
qualifiers:
    Key: gene, Value: example origin spanning gene

